# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
!pip install python-dotenv


In [80]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [8]:
#!pip install pandas

In [21]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

import pandas as pd
df = pd.read_csv("Data.csv")


Saving Data.csv to Data.csv


In [22]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [23]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [25]:
!pip install langchain openai


In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


In [42]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.5)


In [43]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Please provide a detailed and attractive description of the following product: {product}"
)


In [44]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)


In [45]:

product = df['Product'].iloc[0]  # You can try other rows like iloc[1], iloc[2], etc.


In [33]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [49]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])


In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

prompt = ChatPromptTemplate.from_template("Say hello to {name}")
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("Renad"))


Hello Renad! How can I assist you today?


In [51]:
output = chain.run(product)
print(output)


Hello Queen Size Sheet Set! It's nice to meet you.


## SimpleSequentialChain

In [53]:
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(temperature=0.9)


In [54]:
from langchain.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_template(
    "Write a product description for the following item: {product}"
)


# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [56]:

second_prompt = ChatPromptTemplate.from_template(
    "Translate the following product description to French: {input}"
)


# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [57]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
)


In [58]:
product = df['Product'].iloc[1]  # جرّب iloc[2] لاحقًا

output = overall_simple_chain.run(product)
print(output)




> Entering new SimpleSequentialChain chain...
Keep your phone safe and dry with our Waterproof Phone Pouch. Perfect for all your water-based activities like swimming, boating, or beach days, this pouch will protect your phone from water, dust, and sand. The clear window allows you to still use your phone while it's safely sealed inside. The adjustable neck strap ensures that you can keep your phone close by at all times. Don't let water ruin your phone - grab a Waterproof Phone Pouch and enjoy your favorite activities worry-free.
Gardez votre téléphone en sécurité et au sec avec notre Pochette Étanche pour Téléphone. Parfaite pour toutes vos activités aquatiques comme la natation, le bateau ou les journées à la plage, cette pochette protégera votre téléphone de l'eau, de la poussière et du sable. La fenêtre transparente vous permet d'utiliser toujours votre téléphone tout en étant bien scellé à l'intérieur. La sangle ajustable autour du cou assure que vous pouvez garder votre télépho

In [59]:
product = df['Product'].iloc[2]
print(overall_simple_chain.run(product))

product = df['Product'].iloc[3]
print(overall_simple_chain.run(product))




> Entering new SimpleSequentialChain chain...
Introducing the epitome of comfort and luxury - our Luxury Air Mattress. Crafted with the finest materials and designed for ultimate relaxation, this air mattress is perfect for those seeking a peaceful night's sleep wherever they go. 

The plush, velvety surface provides a soft and cozy sleeping surface, while the built-in pump allows for easy inflation and deflation in just minutes. The innovative design evenly distributes weight for maximum support and comfort, ensuring you wake up feeling refreshed and rejuvenated.

Whether you're hosting guests in your home, camping under the stars, or simply need a comfortable bed for your next road trip, our Luxury Air Mattress is the perfect solution. Treat yourself to a restful and luxurious night's sleep every time with our premium air mattress.
Présentation de l'épitome du confort et du luxe - notre matelas pneumatique de luxe. Fabriqué avec les meilleurs matériaux et conçu pour une relaxation 

**Repeat the above twice for different products**

## SequentialChain

In [60]:
from langchain.chains import SequentialChain

In [61]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into Arabic:\n\n{review}"
)


chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="arabic_review"
                    )


In [62]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in one sentence:\n\n{review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )


In [63]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "What language is this review written in?\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="language"
)


In [64]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up message to the customer in {language} thanking them for their review:\n\n\"{summary}\""
)

chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    output_key="followup_message"
)


In [65]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["arabic_review", "summary", "language", "followup_message"],
    verbose=True
)


In [66]:
review = df['Review'].iloc[5]
overall_chain({"review": review})


<ipython-input-66-c86db87991b6>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain({"review": review})




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'arabic_review': 'أجد الطعم سيء. لا تدوم الرغوة، هذا أمر غريب. أشتري نفس الأشياء من المتاجر والطعم أفضل بكثير...\nهل هو دفعة قديمة أم تقليد؟',
 'summary': 'Le reviewer trouve le goût des produits médiocre, la mousse ne tient pas, et il pense que cela pourrait être dû à un vieux lot ou à des contrefaçons.',
 'language': 'It is written in French.',
 'followup_message': "Cher client,\n\nNous vous remercions pour votre retour honnête sur nos produits. Nous sommes désolés d'apprendre que vous n'avez pas été pleinement satisfait de votre expérience. Nous prenons vos commentaires très au sérieux et nous allons enquêter sur la qualité des lots concernés pour nous assurer que cela ne se reproduise plus à l'avenir.\n\nNous sommes reconnaissants pour votre feedback qui nous aide à nous améliorer constamment. Nous espéron

**Repeat the above twice for different products or reviews**

In [68]:
review = df['Review'].iloc[6]
overall_chain({"review": review})

review = df['Review'].iloc[4]
overall_chain({"review": review})




> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


{'review': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'arabic_review': 'أحببت هذا المنتج. ولكن يبدو أنهم يستمرون لبضعة أشهر فقط. كانت الشركة رائعة في استبدال الجهاز الأول (الخلاط يسقط من المقبض ولا يمكن إصلاحه). بعد 4 أشهر فعل الجهاز الثاني نفس الشيء. أستخدم الخلاط للقهوة مرة واحدة في اليوم فقط. ليس بسبب الاستخدام المفرط أو السوء. أنا بشدة بخيبة أمل وسأبحث عن جهاز آخر. وكما أفهم أنهم سوف يستبدلون مرة واحدة فقط. على أي حال، إذا كان لديك واحداً فحظاً سعيداً.',
 'summary': 'The reviewer loved the product but was disappointed that it only lasted a few months and had to be replaced twice due to a faulty frother.',
 '

## Router Chain

In [69]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [5]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [6]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [70]:
llm = ChatOpenAI(temperature=0)

In [71]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [72]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [73]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [74]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [75]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

<ipython-input-75-ae9cc652db54>:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [76]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [77]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [78]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [79]:
chain.run("Who was Napoleon Bonaparte?")
chain.run("How do computers convert code into machine instructions?")




> Entering new MultiPromptChain chain...
History: {'input': 'Who was Napoleon Bonaparte?'}
> Finished chain.


> Entering new MultiPromptChain chain...
computer science: {'input': 'How do computers convert code into machine instructions?'}
> Finished chain.


"Computers convert code into machine instructions through a process called compilation or interpretation. \n\nIn compilation, the source code is translated into machine code by a compiler. The compiler reads the entire source code and translates it into a binary executable file that can be directly executed by the computer's processor.\n\nIn interpretation, the source code is read line by line and translated into machine code on the fly by an interpreter. The interpreter executes the code immediately without the need for a separate compilation step.\n\nBoth compilation and interpretation involve translating high-level code written in languages like C++, Java, or Python into low-level machine code that the computer's processor can understand and execute. This process involves breaking down the code into individual instructions, optimizing the code for efficiency, and handling memory management and other system resources.\n\nOverall, the conversion of code into machine instructions is a 